### About this notebook:

This notebook includes codes that joins planting zone data with temperature/precipitation data. 

Temperature/precipitation data: monthly average of maximum temperature, minimum temperature, and precipitation at city level. 
Planting zone: each US zip code with it's assigned planting zone. 

Planting zone data is left joined with temperature data based on the closest coordinates. 

#### Data dictionary (zipcode_temp_zone.csv)
zip_code: US zip code

zone: planting zone

city: city

state: state

tmax: monthly average maxmium temperature

tmin: monthly average minimum temperature	

prcp: monthly average minimum precipitation

In [64]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import re
import plotly.express as px

In [63]:
temp = pd.read_csv('temp_city_mavg_geo-Copy1.csv', index_col=0)
temp

,month,tmax,tmin,prcp,city_name,ID,Name,Lat,Lon
0,1,25.180000,6.267742,0.016194,Aberdeen,USW00014929,Aberdeen,45.4558,-98.4131
1,2,24.024507,2.581773,0.017259,Aberdeen,USW00014929,Aberdeen,45.4558,-98.4131
2,3,41.161935,20.584516,0.028581,Aberdeen,USW00014929,Aberdeen,45.4558,-98.4131
3,4,55.457333,29.265333,0.048533,Aberdeen,USW00014929,Aberdeen,45.4558,-98.4131
4,5,70.983226,44.136774,0.062065,Aberdeen,USW00014929,Aberdeen,45.4558,-98.4131
...,...,...,...,...,...,...,...,...,...
5407,10,67.499785,46.855140,0.113112,Zaneville,USW00093824,Zaneville,39.9444,-81.8922
5408,11,51.367333,32.136667,0.102267,Zaneville,USW00093824,Zaneville,39.9444,-81.8922
5409,11,51.367333,32.136667,0.102267,Zaneville,USW00093824,Zaneville,39.9444,-81.8922
5410,12,44.502581,27.864516,0.089871,Zaneville,USW00093824,Zaneville,39.9444,-81.8922


In [101]:
zone = pd.read_csv('planting_zone-Copy1.csv', index_col=0,dtype={'zip_code': 'string'})
zone

,zip_code,zone,city,state,latitude,longitude,subzone
0,00705,12,AIBONITO,PR,18.14,-66.26,b
1,00610,12,ANASCO,PR,18.28,-67.14,b
2,00612,12,ARECIBO,PR,18.45,-66.73,b
3,00601,11,ADJUNTAS,PR,18.16,-66.72,b
4,00602,12,AGUADA,PR,18.38,-67.18,b
...,...,...,...,...,...,...,...
29749,30293,8,WOODBURY,GA,32.98,-84.58,a
29750,30188,7,WOODSTOCK,GA,34.10,-84.51,b
29751,30189,7,WOODSTOCK,GA,34.10,-84.51,b
29752,30582,7,YOUNG HARRIS,GA,34.93,-83.84,a


In [102]:
gdf1 = gpd.GeoDataFrame(
    temp, geometry=gpd.points_from_xy(temp['Lon'], temp['Lat']))

gdf2 = gpd.GeoDataFrame(
    zone, geometry=gpd.points_from_xy(zone['longitude'], zone['latitude']))

gdf2['geometry'] = gdf2.geometry.buffer(1)

df = gpd.sjoin(gdf2, gdf1, how="left") 

In [103]:
df

,zip_code,zone,city,state,latitude,longitude,subzone,geometry,index_right,month,tmax,tmin,prcp,city_name,ID,Name,Lat,Lon
0,00705,12,AIBONITO,PR,18.14,-66.26,b,"POLYGON ((-65.26000 18.14000, -65.26482 18.041...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00610,12,ANASCO,PR,18.28,-67.14,b,"POLYGON ((-66.14000 18.28000, -66.14482 18.181...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00612,12,ARECIBO,PR,18.45,-66.73,b,"POLYGON ((-65.73000 18.45000, -65.73482 18.351...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00601,11,ADJUNTAS,PR,18.16,-66.72,b,"POLYGON ((-65.72000 18.16000, -65.72482 18.061...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00602,12,AGUADA,PR,18.38,-67.18,b,"POLYGON ((-66.18000 18.38000, -66.18482 18.281...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29753,30295,8,ZEBULON,GA,33.10,-84.34,a,"POLYGON ((-83.34000 33.10000, -83.34482 33.001...",2875.0,11.0,68.734000,42.480667,0.101533,Macon,USW00003813,Macon,32.6847,-83.6528
29753,30295,8,ZEBULON,GA,33.10,-84.34,a,"POLYGON ((-83.34000 33.10000, -83.34482 33.001...",2876.0,11.0,68.734000,42.480667,0.101533,Macon,USW00003813,Macon,32.6847,-83.6528
29753,30295,8,ZEBULON,GA,33.10,-84.34,a,"POLYGON ((-83.34000 33.10000, -83.34482 33.001...",2877.0,12.0,62.840645,40.332258,0.166516,Macon,USW00003813,Macon,32.6847,-83.6528
29753,30295,8,ZEBULON,GA,33.10,-84.34,a,"POLYGON ((-83.34000 33.10000, -83.34482 33.001...",2878.0,12.0,62.840645,40.332258,0.166516,Macon,USW00003813,Macon,32.6847,-83.6528


In [104]:
df = df.dropna()

In [105]:
test = df.groupby('zip_code').first().reset_index()
test

,zip_code,zone,city,state,latitude,longitude,subzone,geometry,index_right,month,tmax,tmin,prcp,city_name,ID,Name,Lat,Lon
0,01001,6,AGAWAM,MA,42.06,-72.61,a,"POLYGON ((-71.61000 42.06000, -71.61482 41.961...",5328.0,1.0,33.915973,20.381112,0.110558,Worcester,USW00094746,Worcester,42.2706,-71.8731
1,01002,5,AMHERST,MA,42.37,-72.52,b,"POLYGON ((-71.52000 42.37000, -71.52482 42.271...",5328.0,1.0,33.915973,20.381112,0.110558,Worcester,USW00094746,Worcester,42.2706,-71.8731
2,01005,6,BARRE,MA,42.42,-72.10,a,"POLYGON ((-71.10000 42.42000, -71.10482 42.321...",972.0,1.0,34.314839,16.514194,0.090194,Concord,USW00014745,Concord,43.1953,-71.5011
3,01007,5,BELCHERTOWN,MA,42.27,-72.40,b,"POLYGON ((-71.40000 42.27000, -71.40482 42.171...",5328.0,1.0,33.915973,20.381112,0.110558,Worcester,USW00094746,Worcester,42.2706,-71.8731
4,01010,5,BRIMFIELD,MA,42.11,-72.20,b,"POLYGON ((-71.20000 42.11000, -71.20482 42.011...",5328.0,1.0,33.915973,20.381112,0.110558,Worcester,USW00094746,Worcester,42.2706,-71.8731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21277,99362,7,WALLA WALLA,WA,46.11,-118.30,a,"POLYGON ((-117.30000 46.11000, -117.30482 46.0...",3720.0,1.0,42.992258,29.318065,0.053161,Pendleton,USW00024155,Pendleton,45.6906,-118.8528
21278,99371,7,WASHTUCNA,WA,46.82,-118.32,a,"POLYGON ((-117.32000 46.82000, -117.32482 46.7...",4884.0,1.0,42.450323,28.156774,0.032387,Tri Cities,USW00024163,TriCities,46.2667,-119.1167
21279,99401,6,ANATONE,WA,46.08,-117.27,b,"POLYGON ((-116.27000 46.08000, -116.27482 45.9...",2628.0,1.0,42.284516,30.488387,0.031806,Lewiston,USW00024149,Lewiston,46.3747,-117.0156
21280,99402,7,ASOTIN,WA,46.17,-117.16,a,"POLYGON ((-116.16000 46.17000, -116.16482 46.0...",2628.0,1.0,42.284516,30.488387,0.031806,Lewiston,USW00024149,Lewiston,46.3747,-117.0156


In [106]:
test.to_csv('zipcode_temp_zone.csv')